## TensorFlow Linear

In [1]:
import tensorflow as tf

csvFile = 'pima-indians-diabetes.csv'

In [6]:
learning_rate = 0.00005
epoch = 1
batch_size = 1
filename_queue = tf.train.string_input_producer([csvFile])
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
record_defaults = [[1.], [1.], [1.], [1.], [1.], [1.], [1.], [1.], [1]]

features = tf.decode_csv(value, record_defaults=record_defaults)
X_data = features[:-1]
Y_data = features[-1:]
# batch_xs, label = tf.train.shuffle_batch(tensors=[X_data, Y_data], batch_size=batch_size,
#                                          num_threads=1,
#                                          capacity=5000,
#                                          min_after_dequeue=100)
batch_xs, label = tf.train.batch(tensors=[X_data, Y_data], batch_size=batch_size)
label = tf.one_hot(label, depth=2) # 0 => [1,0] 1 => [0,1]
label = tf.reshape(label,[-1,2])
features = len(X_data)
classes = 2
print(type(features), type(classes))

<class 'int'> <class 'int'>


In [28]:
X = tf.placeholder(tf.float32, [None, features], name="input")
Y = tf.placeholder(tf.float32, [None, classes], name="output")

W = tf.Variable(tf.random_normal([features, classes]))
b = tf.Variable(tf.random_normal([classes], name='bias'))
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
print(Y,hypothesis)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.arg_max(hypothesis,1), tf.arg_max(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Tensor("output_5:0", shape=(?, 2), dtype=float32) Tensor("Softmax_5:0", shape=(?, 2), dtype=float32)


In [24]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        for step in range(2001):
            opt, cc = sess.run([optimizer,cost], feed_dict={X:batch_xs.eval(), Y:label.eval()})
    #         print(batch_xs.eval(),label.eval())
            if step%20 ==0:
                atemp = accuracy.eval({X:batch_xs.eval(), Y:label.eval()})
                
                print(step, cc, atemp)
    
#     opt, cc,hy = sess.run([optimizer,cost, hypothesis], feed_dict={X:batch_xs.eval(), Y:label.eval()})
#     print(batch_xs.eval(),W.eval(),hy)
    coord.request_stop()
    coord.join(threads)

0 42.9783 0.0
20 nan 0.0
40 nan 1.0
60 nan 0.0
80 nan 0.0
100 nan 0.0
120 nan 0.0
140 nan 1.0
160 nan 0.0
180 nan 1.0
200 nan 1.0
220 nan 1.0
240 nan 1.0
260 nan 1.0
280 nan 1.0
300 nan 1.0
320 nan 0.0
340 nan 1.0
360 nan 0.0
380 nan 1.0
400 nan 1.0
420 nan 1.0
440 nan 0.0
460 nan 1.0
480 nan 0.0
500 nan 1.0
520 nan 1.0
540 nan 0.0
560 nan 1.0
580 nan 1.0
600 nan 1.0
620 nan 1.0
640 nan 0.0
660 nan 1.0
680 nan 0.0
700 nan 1.0
720 nan 1.0
740 nan 1.0
760 nan 1.0
780 nan 1.0
800 nan 1.0
820 nan 0.0
840 nan 0.0
860 nan 1.0
880 nan 1.0
900 nan 0.0
920 nan 0.0
940 nan 1.0
960 nan 1.0
980 nan 1.0
1000 nan 1.0
1020 nan 1.0
1040 nan 1.0
1060 nan 0.0
1080 nan 1.0
1100 nan 0.0
1120 nan 1.0
1140 nan 0.0
1160 nan 1.0
1180 nan 0.0
1200 nan 0.0
1220 nan 0.0
1240 nan 0.0
1260 nan 1.0
1280 nan 0.0
1300 nan 0.0
1320 nan 1.0
1340 nan 1.0
1360 nan 1.0
1380 nan 1.0
1400 nan 1.0
1420 nan 0.0
1440 nan 1.0
1460 nan 1.0
1480 nan 0.0
1500 nan 1.0
1520 nan 1.0
1540 nan 0.0
1560 nan 0.0
1580 nan 1.0
1600 nan 0.0